In [ ]:
import numpy as np
import scipy.signal
import hfda
from petastorm import make_reader
import matplotlib.pyplot as plt
from scipy import interpolate

In [ ]:
###Mean and variance for a specific interval
def Mean_taux(signal,taux,hprime,h=0):
    if taux+hprime+h > len(signal):
        return np.mean((signal[h+taux:-1]))
    else:
        return np.mean((signal[h+taux:taux+hprime+h]))

def Variance_taux(signal,taux,hprime,h=0):
    if taux+hprime+h > len(signal):
        return np.var((signal[h+taux:-1]))
    else :
        return np.var((signal[h+taux:taux+hprime+h]))

In [ ]:
###I1 and I2:

def I1(c,signal,fs,h,hprime,t0=0):
    if (t0>c or t0<0):
        return "error : t0 is outside the range you want to calculate"
    elif (c>len(signal) or c<0):
        return "error : c is outside the range of your signal"
    else: 
        I1c = 0
        for i in range(t0,int(c*fs)): 
            a =(1/h)*np.abs(Mean_taux(signal,i,hprime,h)-Mean_taux(signal,i,hprime))
            I1c = I1c+a
    return I1c

def I2(c,signal,fs,h,hprime,t0=0):
    if (t0>c or t0<0):
        return "error : t0 is outside the range you want to calculate"
    elif (c>len(signal) or c<0):
        return "error : c is outside the range of your signal"
    else: 
        I2c = 0
        for i in range(t0,int(c*fs)): 
            a =  (1/h)*np.abs(Variance_taux(signal,i,hprime,h)-Variance_taux(signal,i,hprime))
            I2c = I2c+a
    return I2c

In [ ]:
def discrepancies_mean_curve(signal,fs,h,hprime,t0=0):
    c = [i for i in range(1,int(len(signal)/fs),1)]
    I1_val = np.array([0])
    I2_val = np.array([0])
    for j in c:
        I1_val = np.append(I1_val,I1(j,signal,fs,h,hprime,t0))
        I2_val = np.append(I2_val,I2(j,signal,fs,h,hprime,t0))
    c.insert(0,0)
    return I1_val,I2_val,c

In [ ]:
###Test : Get data
path_formatted_glasgow = "/workspaces/maitrise/data/20220902_data_physio_formatted_merged/merged/dataParquet"
path_petastorm = f"file:///{path_formatted_glasgow}"
with make_reader(path_petastorm) as reader:
    for sample in reader:
        data = sample
        break

print(data)




In [ ]:
##Plot data :
def plot_ECG_signal(signal,name,length= data.signal_length,fs = data.sampling_frequency):
     x = np.array(range(0,length))
     x = x/fs     
     fig,ax = plt.subplots(nrows = 1,ncols = 2, figsize = (20,10))
     ax[0].plot(x,signal)
     ax[0].set_title(f"Full signal of Lead {name.decode('utf8')}")
     ax[0].grid()
     ax[1].plot(x,signal)
     ax[1].set_title(f"Close up signal of Lead {name.decode('utf8')}")
     ax[1].grid()
     if len(x) == data.signal_length:
          ax[1].set_xlim([0,3])
     else :
          ax[1].set_xlim([0,x[-1]])
     plt.show()

ECG_signal = data.signal
ECG_lead = data.signal_names
dico_ECG = {}

for i,j in zip(ECG_lead,range(12)):
     dico_ECG[i] = ECG_signal[:,j]
     print(dico_ECG[i].shape)
     plot_ECG_signal(dico_ECG[i],i)

In [ ]:
##Test function : 

h = 100
hprime = 1000
fs = data.sampling_frequency

for i in ECG_lead:
    I1_c,I2_c,c = discrepancies_mean_curve(dico_ECG[i],fs,h,hprime)
    fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
    ax[0].set_title(f"Lead {i.decode('utf8')}")
    ax[0].plot(c,I1_c,"o")
    ax[0].set_xlabel("signal legnth [s]")
    ax[0].set_ylabel("I1 value")
    ax[0].grid()
    ax[1].set_title(f"Lead {i.decode('utf8')}")
    ax[1].plot(c,I2_c,"o")
    ax[1].set_xlabel("signal legnth [s]")
    ax[1].set_ylabel("I2 value")
    ax[1].grid()
    


In [ ]:
###Implementation of Higuchi method 

def Lm_q(signal,m,k,fs=data.sampling_frequency):
    N = len(signal)
    n = np.floor((N-m)/k).astype(np.int64)
    norm  = ((N-1)/(n*k))/(k*(1/fs))
    sum = np.sum(np.abs(np.diff(signal[m::k], n=1)))
    Lmq = (sum*norm)
    return Lmq

def Lq_k(signal,k,fs=data.sampling_frequency):
    calc_L_series = np.frompyfunc(lambda m: Lm_q(signal,m,k,fs), 1, 1)
    L_average = np.average(calc_L_series(np.arange(1, k+1)))
    return L_average


def Dq(signal,kmax,fs=data.sampling_frequency):
    calc_L_average_series = np.frompyfunc(lambda k: Lq_k(signal,k,fs), 1, 1)

    k = np.arange(1, kmax+1)
    L = calc_L_average_series(k).astype(np.float64)

    D, _ = - np.polyfit(np.log2(k), np.log2(L), 1)

    return D




In [ ]:
##Trial and error for determining method for optimal signal length

Global_Dico = {}
Dico_dimension = {}
fs = data.sampling_frequency
for i in ECG_lead:
    signal_lead = dico_ECG[i]
    c = [i for i in range(1,int(len(signal_lead)/fs),1)]
    for boundary in c:
        j =1
        D = np.array([])
        while (j*boundary*fs)<=len(signal_lead):
            sig  = signal_lead[(j-1)*boundary*fs:(j)*boundary*fs]
            D = np.append(D,Dq(sig,20,fs))
            j+=1
        Dico_dimension[f"{boundary} sec"] = D
    Global_Dico[i] = Dico_dimension
    Dico_dimension = {}





In [ ]:
##RMS:

def RMS(tab_val):
    N = len(tab_val)
    square_sum =0
    for j in range(len(tab_val)):
        for i in range(j+1,len(tab_val)):
            square_sum += np.abs(tab_val[j]-tab_val[i])**2
    norm = 1/(N**2-N)
    rms_val = np.sqrt(norm*square_sum)
    return rms_val

h = 100
hprime = 1000
fs = data.sampling_frequency
c_axis = np.linspace(1,int(data.signal_length/fs),1000)

RMS_val = np.array([])
for j in c_axis:
    D = np.array([])
    for i in ECG_lead:
        w = 1
        Ds = np.array([])
        sig = dico_ECG[i]
        while (w*j*fs)<=len(sig):
            sig_c  = sig[int((w-1)*j*fs):int((w)*j*fs)]
            Ds = np.append(Ds,hfda.measure(sig_c,20))
            w+=1
        
        D = np.append(D,np.mean(Ds))
    RMS_val = np.append(RMS_val,RMS(D))

print(RMS_val) 
plt.plot(c_axis,RMS_val)
plt.title("Evolution of RMS value funciton of length of time series")
plt.xlabel("segment length")
plt.ylabel("RMS val")
plt.grid()
plt.show()

In [ ]:
##Graph comme proposer dans etude :

eps1 = np.linspace(1,9.5,100)
eps2 = np.linspace(0.01,0.095,100)
#c_axis = np.linspace(1,int(data.signal_length/fs),1000)
c_axis = {}

x,y = np.meshgrid(eps1,eps2)

for i in ECG_lead:
    sig = dico_ECG[i]
    I1_c,I2_c,c = discrepancies_mean_curve(sig,fs,h,hprime)
    f1 = np.polyfit(I1_c,c,1)
    f2 = np.polyfit(I2_c,c,1)
    l1 = np.poly1d(f1)
    l2 = np.poly1d(f2)
    c1 = l1(eps1)
    c2 = l2(eps2)
    c1x,c2y = np.meshgrid(c1,c2)
    print(c1x)
    z = np.empty((len(c1),len(c2)))
    for w in range(len(c1)):
        for j in range(len(c2)):
            z[w][j] = np.min(c1x[w][j],c2y[w][j])
    c_axis[i] = z

print(c_axis)


In [ ]:
###Comparison TSD between differente lead:

segment_length = 1.5
D_lead = {}
h = 100
hprime = 1000
fs = data.sampling_frequency
for i in ECG_lead:
    w=1
    Ds = np.array([])
    sig = dico_ECG[i]
    while (w*segment_length*fs)<=len(sig):
        sig_c  = sig[int((w-1)*segment_length*fs):int((w)*segment_length*fs)]
        Ds = np.append(Ds,hfda.measure(sig_c,20))
        w+=1
    D_lead[i] = Ds

w_length = [w*segment_length for w in range(0,int((data.signal_length/fs)*(1/segment_length)))]

for i in ECG_lead:
    if i.decode('utf8') == "V6":
        plt.plot(w_length,D_lead[i],"--r",label = i.decode('utf8'))
    elif i.decode('utf8') == "V5":
        plt.plot(w_length,D_lead[i],"--b",label = i.decode('utf8'))
    else :
        plt.plot(w_length,D_lead[i],label = i.decode('utf8'))
    plt.xlabel(" Time interval")
    plt.ylabel("TSD value")
plt.legend(loc = "best", bbox_to_anchor=(1, 1))
plt.grid()
plt.show()


In [ ]:
###Function for TF phase shuffling (algo 1 of the article : "Testing for nonlinearity in time series:
#the method of surrogate data")
from random import uniform
from scipy.fftpack import rfft,irfft,fft, ifft

def Random_phase_shuffling(signal):
    fft_signal = rfft(signal)
    phase_fs = np.arctan2(fft_signal[2::2],fft_signal[1:-1:2])
    mag = np.sqrt((fft_signal[1:-1:2])**2+(fft_signal[2::2])**2)
    ##phase shuffler:
    rng_phase = phase_fs.copy()
    np.random.shuffle(rng_phase)
    ##new signal : 
    fsrp = mag[:, np.newaxis] * np.c_[np.cos(rng_phase), np.sin(rng_phase)]
    fsrp = np.r_[fft_signal[0], fsrp.ravel(), fft_signal[-1]]
    tsr = irfft(fsrp)
    return tsr

sig = dico_ECG[ECG_lead[6]]
new_sig = Random_phase_shuffling(sig)

fig,ax = plt.subplots(1,2,figsize = (20,10))
ax[0].plot(sig,alpha =0.6)
ax[0].set_title("Old lead")
ax[0].plot(new_sig, alpha =0.6)
ax[1].set_title("noisy lead")



In [ ]:
###Trial noise : 
segment_length = 1.5
D_lead = {}
h = 100
hprime = 1000
fs = data.sampling_frequency
Noisy_ECG = {}
for i in ECG_lead:
    Noisy_ECG[i] = Random_phase_shuffling(dico_ECG[i])


for i in ECG_lead:
    w=1
    Ds = np.array([])
    sig = Noisy_ECG[i]
    while (w*segment_length*fs)<=len(sig):
        sig_c  = sig[int((w-1)*segment_length*fs):int((w)*segment_length*fs)]
        Ds = np.append(Ds,Dq(sig_c,20,fs))
        w+=1
    D_lead[i] = Ds

w_length = [w*segment_length for w in range(0,int((data.signal_length/fs)*(1/segment_length)))]

for i in ECG_lead:
    if i.decode('utf8') == "V6":
        plt.plot(w_length,D_lead[i],"--r",label = i.decode('utf8'))
    elif i.decode('utf8') == "V5":
        plt.plot(w_length,D_lead[i],"--b",label = i.decode('utf8'))
    else :
        plt.plot(w_length,D_lead[i],label = i.decode('utf8'))
    plt.xlabel(" Time interval")
    plt.ylabel("TSD value")
plt.legend(loc = "best", bbox_to_anchor=(1, 1))
plt.grid()
plt.show()
